# Decorator
A decorator is just a function that takes another function as input, extends its behavoir, and return the extended function.


##Function as first-class object
In Python, everything is an object.  This certainly includes functions.  A function is also an object. It can assigned to a variable, can be passed into another function as an argument, and can be returned by another function.

In [ ]:
# function can be assign to a variable
def say_hello():
  print('Hello')

say_hello()
a = say_hello

a()

Hello
Hello


In [1]:
# function as input argument of another function
a = [1, 3, 5]
b = [2, 4, 6]
def do_something(f, lx, ly):
  l = []
  for i, j in zip(lx, ly):
    l.append(f(i, j))
  return l

def add(x, y):
  return x+y
def sub(x, y):
  return x-y

r = do_something(add, a, b)
print(r)
s = do_something(sub, a, b)
print(s)

[3, 7, 11]
[-1, -1, -1]


In [ ]:
# functions can be returned as result of a function
def outer():
  def inner():
    print('I am inner')
  return inner

a = outer()
a()

I am inner


## Higher order function and Inner function
Functions that take other functions as arguments are also called **higher order functions**.

Functions that are defined inside another function are also called **inner functions**

## Callables 
Any object that implements the ``__call__()`` method can be called, and is called a **callable**.

Functions and methods of a class are callables.




## The most basic decorator
Basically, a decorator is a callable that takes in a function as argument, adds some functionality, and returns it.

The structure of a decorator is almost always such that:
1. The decorator function takes a function as input argument
2. The decorator contains an inner function
3. The inner function do something interesting, and then do something else by calling the input function.
4. The decorator function returns the inner function

In [2]:
def add_capability(func):
  def inner():
    print('I added capability to this function')
    func()
  return inner

def say_hello():
  print('Hello')

new_hello = add_capability(say_hello)

new_hello()

I added capability to this function
Hello


## Common practice when decorating a function
The common practice when decorating a function is to assigned the decorated function back to the original function name.

It make the code more convenient.  However, it is often also a source of confusion for beginners of python.

In [ ]:
def in_the_morning():
  print('Good morning')

in_the_morning()

in_the_morning = add_capability(in_the_morning)
in_the_morning()

## Decorating functions with parameters

In [ ]:
# This will cause an exception, since a and b have different element types
a = ['a', 'b']
b = [3, 4, 5]
c = [3, 5, 6]
do_something(sub, a, b)

In [3]:
# Decorating functions with parameters
def check_types(func):
  def inner(f, a, b):
    if not(type(a[0])==type(b[0])):
      print('The two input lists must have same element types')
      return
    return func(f, a, b)
  return inner

def do_something(f, lx, ly):
  l = []
  for i, j in zip(lx, ly):
    l.append(f(i, j))
  return l

def add(x, y):
  return x+y
def sub(x, y):
  return x-y

do_something = check_types(do_something)

a = ['a', 'b']
b = [3, 4, 5]
c = [3, 5, 6]
do_something(sub, a, b)
do_something(sub, c, b)

The two input lists must have same element types


[0, 1, 1]

## Shorthand for decorators

Python provides the following shorthand for decorators:

```
# assume deco is already defined
@deco
def f(...):
  .....
```
This has the same effect as:
```
# assume deco is already defined
def f(...):
  .....
f = deco(f)
```


In [ ]:
def divide(a, b):
  return a/b

print(divide(3, 5))
print(divide(3, 0))

0.6


ZeroDivisionError: ignored

In [ ]:
def check_divident(func):
  def inner(a, b):
    if b == 0:
      print('Cannot divide by zero')
      return
    return func(a, b)
  return inner

@check_divident
def divide(a, b):
  return a/b

# We have decorate the function "divide" with the capability to check divident
print(divide(2, 5))
print(divide(7, 0))


0.4
Cannot divide by zero
None


##Decorating function of with any parameters


In [ ]:
def works_for_all(func):
  def inner(*args, **kwargs):
    print('I can decorate any function')
    return func(*args, **kwargs)
  return inner

@works_for_all
def divide(a, b):
  return a/b

@works_for_all
def hello(name):
  print("Hello, ", name)

result = divide(3, 6)
print(result)

hello('John')


I can decorate any function
0.5
I can decorate any function
Hello,  John


# Chaining multiple decorators

We can chain multiple decorators together to decorate a single function.

Note the order of decorators and their effect on the decorated function.

In [4]:
def star(func):
  def inner(*args, **kwargs):
    return '***'+func(*args, **kwargs)+'***'
  return inner

def dash(func):
  def inner(*args, **kwargs):
    return '---'+func(*args, **kwargs)+'---'
  return inner

def check_string(func):
  def inner(*args, **kwargs):
    for arg in args:
      if type(arg) != str:
        print('only accept string arguments')
        return 'error'
    return func(*args, **kwargs)
  return inner

@dash
@star
@check_string
def concat(a, b):
  return a+b

print(concat('good ', 'morning'))
print(concat('good ', 33))

---***good morning***---
only accept string arguments
---***error***---


## 口訣
大, 吃小, 吐中, 稱小

## Advanced topic: identify of the function



In [ ]:
# A function can know about it's own identify
def concat(a, b):
  return a+b

print(concat.__name__)
a = concat
print(a.__name__)

concat
concat


In [ ]:
# After we decorate a function, 
# the identify becomes the inner function of the decorate functoin
# This is not what we want, because it is not useful.
def check_string(func):
  def inner(*args, **kwargs):
    for arg in args:
      if type(arg) != str:
        print('only accept string arguments')
        return 'error'
    return func(*args, **kwargs)
  return inner

@check_string
def concat(a, b):
  return a+b

a = concat
print(concat.__name__)
print(a.__name__)

inner
inner


In [ ]:
# We can use the 'functools.wraps(func)' to assign the identify we want 
# to the newly decorated function
import functools
def check_string(func):
  # yes, we use a decorator inside a decorator definition!
  @functools.wraps(func)
  def inner(*args, **kwargs):
    for arg in args:
      if type(arg) != str:
        print('only accept string arguments')
        return 'error'
    return func(*args, **kwargs)
  return inner

@check_string
def concat(a, b):
  return a+b

a = concat
print(concat.__name__)
print(a.__name__)

concat
concat
